In [1]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import heapq

def dijkstra(adj_matrix, start_city, end_city):
    num_cities = len(adj_matrix)
    distances = [sys.maxsize] * num_cities
    visited = [False] * num_cities
    previous = [None] * num_cities

    distances[start_city] = 0

    for _ in range(num_cities):
        min_dist = sys.maxsize
        current_city = None

        for city in range(num_cities):
            if not visited[city] and distances[city] < min_dist:
                min_dist = distances[city]
                current_city = city

        visited[current_city] = True

        if current_city == end_city:
            break

        for neighbor in range(num_cities):
            if (
                not visited[neighbor]
                and adj_matrix[current_city][neighbor] != 0
                and distances[current_city] + adj_matrix[current_city][neighbor] < distances[neighbor]
            ):
                distances[neighbor] = distances[current_city] + adj_matrix[current_city][neighbor]
                previous[neighbor] = current_city

    if distances[end_city] == sys.maxsize:
        return "No path found."

    path = []
    current_city = end_city
    while current_city is not None:
        path.append(current_city)
        current_city = previous[current_city]

    path.reverse()
    return path, distances[end_city]

In [78]:
num_villes = 500
NumTruck = 3

def generate_weighted_adjacency_matrix(n):
    # Générer une matrice d'adjacence vide
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = random.randint(1, 1000)  # Poids du lien entre deux villes
                adjacency_matrix[i][j] = weight 
                adjacency_matrix[j][i] = weight
                

    return adjacency_matrix

adj_matrix = generate_weighted_adjacency_matrix(num_villes)

population = []
truckJourneys = []
visitedCities = []

initCity = random.randint(1,num_villes)

numValidSol = 12

for n in range(numValidSol):
    for i in range(NumTruck):
        truckJourneys.append([initCity])
    visitedCities.append(initCity)
    print("Initial cities : ", truckJourneys)

    while len(visitedCities) < num_villes:
        for j in range(NumTruck):
            curTruck = j
            curCity = truckJourneys[curTruck][-1]
            neighborsCurCity = adj_matrix[curCity]
            CityFound = False
            non_zero_indices = [i for i, val in enumerate(neighborsCurCity) if val != 0 and i not in visitedCities]
            if all(item in visitedCities for item in non_zero_indices):
                non_zero_count = len([x for x in neighborsCurCity if x != 0])
                probabilities = [0 if x == 0 else 1 / non_zero_count for x in neighborsCurCity]
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
            else:
                non_zero_indices = len([i for i, val in enumerate(neighborsCurCity) if val != 0 and i not in visitedCities])
                probabilities = [1 / non_zero_indices if val != 0 and i not in visitedCities else 0 for i, val in enumerate(neighborsCurCity)]
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
                visitedCities.append(selectedCity)

    for j in range(len(truckJourneys)):
        result = dijkstra(adj_matrix, truckJourneys[j][-1], truckJourneys[j][0])
        path, distance = result
        truckJourneys[j] = truckJourneys[j] + path[1:]

    #for j in range(len(truckJourneys)):
        #print("Truck Journey ", j, " : ", truckJourneys[j])
    
    population.append(truckJourneys)
    
    truckJourneys = []
    visitedCities = []

print("finished")

Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
Initial cities :  [[328], [328], [328]]
finished


In [80]:
# Function to calculate the fitness of an individual
def calculate_fitness(individual):
    distances = []
    for path in individual:
        path_distance = 0
        for i in range(len(path) - 1):
            start_city = path[i]
            end_city = path[i + 1]
            path_distance += adj_matrix[start_city][end_city]
        distances.append(path_distance)
    difference = max(distances) - min(distances)
    return difference

for i in range(len(population)):
    fitness = calculate_fitness(population[i])
    print(fitness)

3186
2787
2806
5667
8715
3173
6235
661
4406
3402
9378
7048


## Algorithme génétique

In [81]:
# Constants
POPULATION_SIZE = numValidSol
NUM_GENERATIONS = 500
MUTATION_RATE = 1

In [82]:
def repair_non_neighboring_cities(path, index):
    if index+1 <= len(path)-1:
        city1 = path[index]
        city2 = path[index+1]
        if city1 == city2:
            path.pop(index)
        elif adj_matrix[city1][city2] == 0:
            result = dijkstra(adj_matrix, city1, city2)
            path_to_add, distance = result
            index_where_add = index+1
            for i in path_to_add:
                path.insert(index_where_add, i)
                index_where_add += 1
    return path

In [83]:
def repair_missing_cities(individual):
    visited_cities = set([city for path in individual for city in path])
    missing_cities = set(range(len(adj_matrix))) - visited_cities
    if len(missing_cities) != 0:
        while len(missing_cities) > 0:
            missing_city = missing_cities.pop()
            for path in individual:
                for i in range(len(path)-1):
                    if adj_matrix[missing_city][i] != 0 and adj_matrix[missing_city][i+1] != 0:
                        path.insert(i+1, missing_city)
                        missing_city = -1
                        break
                if missing_city == -1:
                    break
            if missing_city != -1:
                neighborCities = [i for i, val in enumerate(adj_matrix[missing_city]) if val != 0 and i not in missing_cities]
                neighborCity = random.choice(neighborCities)
                for path in individual:
                    try:
                        indexneighborCity = path.index(neighborCity)
                        path.insert(indexneighborCity+1, missing_city)
                        result = dijkstra(adj_matrix, indexneighborCity+1, indexneighborCity+2)
                        path, distance = result
                        path_to_add = path[1:-1]
                        index_to_add = indexneighborCity+2
                        for i in path_to_add:
                            path.insert(index_to_add, i)
                            index_to_add += 1 
                    except:
                        pass
    return individual

In [84]:
# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = [[] for _ in range(len(parent1))]
    for i in range(len(parent1)):
        cut_point = random.randint(0, len(parent1[i]) - 1)
        child[i] = parent1[i][:cut_point] + parent2[i][cut_point:]
        child[i] = repair_non_neighboring_cities(child[i], cut_point)

    child = repair_missing_cities(child)  # Repair the child to ensure it visits all cities
    return child

In [85]:
# Function to perform mutation on an individual
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            swap_indices = random.sample(range(1, len(individual[i]) - 1), 2)
            individual[i][swap_indices[0]], individual[i][swap_indices[1]] = \
                individual[i][swap_indices[1]], individual[i][swap_indices[0]]
            for index in swap_indices:
                repair_non_neighboring_cities(individual[i], index)
                if index > 0:
                    repair_non_neighboring_cities(individual[i], index-1)
    return individual

In [86]:
# Main genetic algorithm loop
for generation in range(NUM_GENERATIONS):
    # Calculate fitness for each individual
    fitness_scores = [calculate_fitness(individual) for individual in population]
    
    # Find the best individual in the population
    best_fitness = min(fitness_scores)
    best_individual = population[fitness_scores.index(best_fitness)]
    
    print(f"Generation {generation + 1}: Best Distance = {best_fitness}")
    
    # Select parents for crossover using tournament selection
    parents = []
    for _ in range(POPULATION_SIZE // 2):
        tournament = random.sample(range(POPULATION_SIZE), 5)
        tournament_fitness = [fitness_scores[i] for i in tournament]
        winner = tournament[tournament_fitness.index(min(tournament_fitness))]
        parents.append(population[winner])
    
    # Create new population through crossover and mutation
    new_population = []
    for i in range(0, len(parents), 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]
        child1 = crossover(parent1, parent2)
        child2 = crossover(parent2, parent1)
        new_population.extend([mutate(child1), mutate(child2)])
        
    population.extend(new_population)
    fitness_scores.extend([calculate_fitness(individual) for individual in new_population])

    
    # Combine the elements of lists population and fitness_scores
    combined = zip(population, fitness_scores)

    # Sort the pairs based on the values in list fitness_scores
    sorted_pairs = sorted(combined, key=lambda x: x[1])

    # Extract the desired values
    population = [pair[0] for pair in sorted_pairs[:POPULATION_SIZE]]
    fitness_scores = [pair[1] for pair in sorted_pairs[:POPULATION_SIZE]]
    
    
# Print the best solution
best_fitness = min(fitness_scores)
best_individual = population[fitness_scores.index(best_fitness)]
print(f"\nBest Solution: {best_individual}")
print(f"Best Distance: {best_fitness}")

Generation 1: Best Distance = 661
Generation 2: Best Distance = 661
Generation 3: Best Distance = 661
Generation 4: Best Distance = 661
Generation 5: Best Distance = 661
Generation 6: Best Distance = 661
Generation 7: Best Distance = 661
Generation 8: Best Distance = 661
Generation 9: Best Distance = 661
Generation 10: Best Distance = 661
Generation 11: Best Distance = 661
Generation 12: Best Distance = 661
Generation 13: Best Distance = 661
Generation 14: Best Distance = 661
Generation 15: Best Distance = 661
Generation 16: Best Distance = 661
Generation 17: Best Distance = 661
Generation 18: Best Distance = 661
Generation 19: Best Distance = 661
Generation 20: Best Distance = 661
Generation 21: Best Distance = 661
Generation 22: Best Distance = 661
Generation 23: Best Distance = 661
Generation 24: Best Distance = 661
Generation 25: Best Distance = 661
Generation 26: Best Distance = 661
Generation 27: Best Distance = 661
Generation 28: Best Distance = 661
Generation 29: Best Distance 

Generation 232: Best Distance = 404
Generation 233: Best Distance = 404
Generation 234: Best Distance = 404
Generation 235: Best Distance = 51
Generation 236: Best Distance = 51
Generation 237: Best Distance = 51
Generation 238: Best Distance = 51
Generation 239: Best Distance = 51
Generation 240: Best Distance = 51
Generation 241: Best Distance = 51
Generation 242: Best Distance = 51
Generation 243: Best Distance = 51
Generation 244: Best Distance = 51
Generation 245: Best Distance = 51
Generation 246: Best Distance = 51
Generation 247: Best Distance = 51
Generation 248: Best Distance = 51
Generation 249: Best Distance = 51
Generation 250: Best Distance = 51
Generation 251: Best Distance = 51
Generation 252: Best Distance = 51
Generation 253: Best Distance = 51
Generation 254: Best Distance = 51
Generation 255: Best Distance = 51
Generation 256: Best Distance = 51
Generation 257: Best Distance = 51
Generation 258: Best Distance = 51
Generation 259: Best Distance = 51
Generation 260: B

Generation 466: Best Distance = 20
Generation 467: Best Distance = 20
Generation 468: Best Distance = 20
Generation 469: Best Distance = 20
Generation 470: Best Distance = 20
Generation 471: Best Distance = 20
Generation 472: Best Distance = 20
Generation 473: Best Distance = 20
Generation 474: Best Distance = 20
Generation 475: Best Distance = 20
Generation 476: Best Distance = 20
Generation 477: Best Distance = 20
Generation 478: Best Distance = 20
Generation 479: Best Distance = 20
Generation 480: Best Distance = 20
Generation 481: Best Distance = 20
Generation 482: Best Distance = 20
Generation 483: Best Distance = 20
Generation 484: Best Distance = 20
Generation 485: Best Distance = 20
Generation 486: Best Distance = 20
Generation 487: Best Distance = 20
Generation 488: Best Distance = 20
Generation 489: Best Distance = 20
Generation 490: Best Distance = 20
Generation 491: Best Distance = 20
Generation 492: Best Distance = 20
Generation 493: Best Distance = 20
Generation 494: Best

In [42]:
a = [1, 2, 3, 4, 5, 6]
b = [100, 200, 300, 400, 500, 600]

# Find the index of the smallest value in a
min_index = min(enumerate(a), key=lambda x: x[1])[0]

# Extract the three elements from b
result = b[min_index:min_index + 3]

print(result)  # Output: [100, 200, 300]

[100, 200, 300]


In [50]:
a = [1, 2, 3, 4, 5, 6]
b = a.index(1)
print(b)

0


In [275]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [50, 60, 80, 100, 200, 30, 50, 12, 30, 400]

# Combine the elements of lists a and b
combined = zip(a, b)

# Sort the pairs based on the values in list b
sorted_pairs = sorted(combined, key=lambda x: x[1])

# Extract the desired values
c = [pair[0] for pair in sorted_pairs[:6]]
d = [pair[1] for pair in sorted_pairs[:6]]

print("c:", c)
print("d:", d)

c: [8, 6, 9, 1, 7, 2]
d: [12, 30, 30, 50, 50, 60]


In [127]:
a = 0
b=0
while b < 10:
    b += 1
    print(b)
    for i in range(3):
        for j in range(10):
            print("Le nombre est : ", j)
            if j == 5:
                a = -1
                print("boucle j quittée")
                break
        if a == -1:
            print("boucle i quittée")
            break
    print("a")

1
Le nombre est :  0
Le nombre est :  1
Le nombre est :  2
Le nombre est :  3
Le nombre est :  4
Le nombre est :  5
boucle j quittée
boucle i quittée
a
2
Le nombre est :  0
Le nombre est :  1
Le nombre est :  2
Le nombre est :  3
Le nombre est :  4
Le nombre est :  5
boucle j quittée
boucle i quittée
a
3
Le nombre est :  0
Le nombre est :  1
Le nombre est :  2
Le nombre est :  3
Le nombre est :  4
Le nombre est :  5
boucle j quittée
boucle i quittée
a
4
Le nombre est :  0
Le nombre est :  1
Le nombre est :  2
Le nombre est :  3
Le nombre est :  4
Le nombre est :  5
boucle j quittée
boucle i quittée
a
5
Le nombre est :  0
Le nombre est :  1
Le nombre est :  2
Le nombre est :  3
Le nombre est :  4
Le nombre est :  5
boucle j quittée
boucle i quittée
a
6
Le nombre est :  0
Le nombre est :  1
Le nombre est :  2
Le nombre est :  3
Le nombre est :  4
Le nombre est :  5
boucle j quittée
boucle i quittée
a
7
Le nombre est :  0
Le nombre est :  1
Le nombre est :  2
Le nombre est :  3
Le nombre 

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

num_villes = 50
NumTruck = 3

def generate_weighted_adjacency_matrix(n):
    # Générer une matrice d'adjacence vide
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = random.randint(1, 1000)  # Poids du lien entre deux villes
                adjacency_matrix[i][j] = weight 
                adjacency_matrix[j][i] = weight
                

    return adjacency_matrix

adj_matrix = generate_weighted_adjacency_matrix(num_villes)

population = []

# initialise list of visited cities
visitedCities = [[] for i in range(NumTruck)]

# Determine initial cities (can't have 2 times the same)
truckJourneys = []

numValidSol = 10

for n in range(numValidSol):
    for i in range(NumTruck):
        initCity = random.randint(1,num_villes)
        while initCity in truckJourneys:
            initCity = random.randint(1,num_villes)
        truckJourneys.append([initCity])
        visitedCities[i].append(initCity)
    print("Initial cities : ", truckJourneys)

    truckJourneysFinished = 0

    while truckJourneysFinished != NumTruck:
        truckJourneysFinished = 0
        for j in range(NumTruck):
            if len(visitedCities[j]) != num_villes:
                curTruck = j
                curCity = truckJourneys[curTruck][-1]
                neighborsCurCity = adj_matrix[curCity]
                closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                CityFound = False
                minValue = 0
                while CityFound == False:
                    if max(neighborsCurCity) == minValue:
                        closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                        if truckJourneys[curTruck][-2] == closerCity:
                            while neighborsCurCity[closerCity] == 0 or closerCity == truckJourneys[curTruck][-2]:
                                closerCity = neighborsCurCity.index(random.choice(neighborsCurCity))
                            truckJourneys[curTruck].append(closerCity)
                            CityFound = True
                        else:
                            closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                            truckJourneys[curTruck].append(closerCity)
                            CityFound = True
                    else:
                        closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > minValue))
                        if closerCity in visitedCities[j]:
                            minValue = neighborsCurCity[closerCity]
                        else:
                            truckJourneys[curTruck].append(closerCity)
                            visitedCities[j].append(closerCity)
                            CityFound = True
        for k in range(NumTruck):
            if len(visitedCities[k]) == num_villes:
                truckJourneysFinished += 1
    
    for j in range(len(truckJourneys)):
        result = dijkstra(adj_matrix, truckJourneys[j][-1], truckJourneys[j][0])
        path, distance = result
        truckJourneys[j] = truckJourneys[j] + path
    
    for j in range(len(visitedCities)):
        print("Truck Journey ", j, " : ", truckJourneys[j])
    
    for j in range(len(visitedCities)):
        print("With Duplicate", len(truckJourneys[j]))

    for j in range(len(visitedCities)):
        print("Without Duplicate", len(set(truckJourneys[j])))
        
    population.append(truckJourneys)
    visitedCities = [[] for i in range(NumTruck)]
    truckJourneys = []

print(population)
print(len(population))

In [ ]:
# Function to calculate the fitness of an individual
def calculate_fitness(individual):
    total_distance = 0
    for path in individual:
        path_distance = 0
        for i in range(len(path) - 1):
            start_city = path[i]
            end_city = path[i + 1]
            path_distance += adj_matrix[start_city][end_city]
        total_distance += path_distance
    return total_distance

fitness = calculate_fitness(population[0])
print(fitness)

In [ ]:
# Function to repair an individual to ensure it visits all cities
def repair_individual(individual):
    visited_cities = set([city for path in individual for city in path])
    missing_cities = set(range(len(adj_matrix))) - visited_cities
    dijkstra_path_to_add = []
    dijkstra_location_path_to_add = []
    for path in individual:
        for i in range(len(path)-1):
            if adj_matrix[path[i]][path[i+1]] == 0:
                result = dijkstra(adj_matrix, path[i], path[i+1])
                path_to_add, distance = result
                dijkstra_path_to_add.append(path_to_add[1:-1])
                dijkstra_location_path_to_add.append(i+1)

        while len(dijkstra_path_to_add) > 0:
            path_to_add = dijkstra_path_to_add.pop(0)
            index_where_add = dijkstra_location_path_to_add.pop(0)
            for i in path_to_add:
                path.insert(index_where_add, i)
                index_where_add += 1
                dijkstra_location_path_to_add = [x+1 for x in dijkstra_location_path_to_add]
                
    if len(missing_cities) != 0:
        while len(missing_cities) > 0:
            missing_city = missing_cities.pop()
            for path in individual:
                for i in range(len(path)-1):
                    if adj_matrix[missing_city][i] != 0 and adj_matrix[missing_city][i+1] != 0:
                        path.insert(i+1, missing_city)
                        missing_city = -1
                        break
                if missing_city == -1:
                    break
            if missing_city != -1:
                neighborCities = [i for i, val in enumerate(adj_matrix[missing_city]) if val != 0 and i not in missing_cities]
                neighborCity = random.choice(neighborCities)
                for path in individual:
                    try:
                        indexneighborCity = path.index(neighborCity)
                        path.insert(indexneighborCity+1, missing_city)
                        result = dijkstra(adj_matrix, indexneighborCity+1, indexneighborCity+2)
                        path, distance = result
                        path_to_add = path[1:-1]
                        index_to_add = indexneighborCity+2
                        for i in path_to_add:
                            path.insert(index_to_add, i)
                            index_to_add += 1 
                    except:
                        pass
    return individual